# V2 Muestra de 76343 con 2 esqueletos

In [2]:
import json
import os
import random
import numpy as np
from PIL import Image

# Ruta a las imágenes originales y las anotaciones
images_path = os.path.abspath('./mnt/Dataset/images/')
annotations_file = os.path.abspath('./mnt/Dataset/annotations/annotations.json')

# Ruta para guardar las imágenes y anotaciones preprocesadas
output_annotations_dir = os.path.abspath('./mnt/V2/annotations')
output_images_dir = os.path.abspath('./mnt/V2/images')

# Crear los directorios si no existen
if not os.path.exists(output_annotations_dir):
    os.makedirs(output_annotations_dir)

if not os.path.exists(output_images_dir):
    os.makedirs(output_images_dir)

# Leer las anotaciones del archivo JSON
with open(annotations_file, 'r') as f:
    annotations = json.load(f)

# Función para normalizar coordenadas
def normalize_pose(pose, image_width, image_height):
    normalized_pose = []
    for point in pose:
        x, y, c = point
        normalized_x = x / image_width
        normalized_y = y / image_height
        normalized_pose.append([normalized_x, normalized_y, c])
    return normalized_pose

# Crear lista de datos preprocesados
data = []

# Procesar las anotaciones y las imágenes
for annotation in annotations:
    # Si falta pose1 o pose2, saltar esta anotación
    if 'pose1' not in annotation or 'pose2' not in annotation:
        continue
    
    image_name = annotation['image'] + '.jpg'  # El nombre de la imagen
    image_path = os.path.join(images_path, image_name)

    if not os.path.exists(image_path):
        continue  # Si la imagen no existe, pasar a la siguiente

    # Abrir imagen para obtener dimensiones
    image = Image.open(image_path)
    width, height = image.size

    # Extraer y normalizar las poses
    pose1 = annotation.get('pose1')
    pose2 = annotation.get('pose2')
    
    # Normalizar las poses si están presentes
    pose1 = normalize_pose(pose1, width, height)
    pose2 = normalize_pose(pose2, width, height)

    # Extraer la posición de combate
    position = annotation['position']

    # Redimensionar la imagen a 256x256
    resized_image = image.resize((256, 256))

    # Agregar los datos a la lista (imagen, pose1, pose2, posición)
    data.append({
        'image': resized_image,
        'pose1': pose1,
        'pose2': pose2,
        'position': position
    })


# Guardar las imágenes redimensionadas en el directorio output_images_dir
for idx, item in enumerate(data):
    # Guardar la imagen
    image = item['image']
    image.save(os.path.join(output_images_dir, f'{str(idx+1).zfill(7)}.jpg'))  # Nombre con formato 0000001.jpg

# Crear un archivo JSON con las anotaciones correspondientes
output_annotations = []
for idx, item in enumerate(data):
    output_annotations.append({
        'Image': f'{str(idx+1).zfill(7)}',  # Mantener el nombre formateado
        'Pose1': item['pose1'],
        'Pose2': item['pose2'],
        'Position': item['position']
    })

# Guardar las anotaciones en un archivo JSON en output_annotations_dir
output_json_path = os.path.join(output_annotations_dir, 'annotations_preprocessed.json')
with open(output_json_path, 'w') as f:
    json.dump(output_annotations, f)

print(f'Proceso completado. Se han guardado {len(data)} imágenes en {output_images_dir} y las anotaciones en {output_annotations_dir}.')


Proceso completado. Se han guardado 76343 imágenes en d:\yolo-proyect\TFG\mnt\V2\images y las anotaciones en d:\yolo-proyect\TFG\mnt\V2\annotations.
